## Investigate UCI retail data

this notebook investigates the dataset from UCI, and summarizes key parts of it, that should be handled in data cleaning
(it assumes a time series forecasting project will be conducted on the data)

In [ ]:
import sqlite3
import pandas as pd

